In [1]:
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcol
import numpy as np
import datetime as dt

In [2]:
target_variable='t_pot'

In [18]:
    obs=xr.open_dataset('/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/StatDownOc/output/ctdObs/ctd_obs_ds.nc')
    obs=obs[[target_variable]]
    obsmask=~np.isnan(obs[target_variable])

In [35]:
ds_input0=xr.open_dataset('/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/StatDownOc/output/griddedROMS.nc')

In [37]:
bathymetry=ds_input0.bathy.values
ds_input=ds_input0[[target_variable]]
ds_target=ds_input.copy()

In [ ]:
perm=np.random.permutation(len(obs.time))
perm=np.concatenate((perm,perm),axis=0)
omask=obsmask.isel(t_ind=perm[:len(ds_input.time)]).values
ds_input=ds_input*omask

In [22]:
ds_input

<xarray.Dataset>
Dimensions:  (x: 38, z: 800, time: 312)
Coordinates:
  * x        (x) int64 0 1 2 3 4 5 6 7 8 9 10 ... 28 29 30 31 32 33 34 35 36 37
    lat      (x) float64 ...
    lon      (x) float64 ...
  * z        (z) float64 2.5 7.5 12.5 17.5 ... 3.988e+03 3.992e+03 3.998e+03
Dimensions without coordinates: time
Data variables:
    t_pot    (time, z, x) float64 9.627 0.0 9.693 0.0 0.0 ... 0.0 0.0 0.0 0.0

In [23]:
ds_input.t_pot

<xarray.DataArray 't_pot' (time: 312, z: 800, x: 38)>
array([[[9.62738752, 0.        , 9.69318342, ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.        , 0.        , 9.69318342, ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
...
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.        , 0.        , 9.69318342, ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]]])
Coordinates:
  * x        (x) int64 0 1 2 3 4 5 6 7 8 9 10 ... 28 29 30 31 32 33 34 35 36 37
    lat      (x) float64 ...
    lon      (x) float64 ...
  * z        (z) float64 2.5 7.5 12.5 17.5 ... 3.988e+03 3.992e+03 3.998e+03
Dimensions without coordinates: time

In [26]:
ds_target.t_pot.values

array([[[ 9.62738752,  9.60155582,  9.69318342, ..., 14.89295089,
         14.74384667, 14.61212   ],
        [ 9.57817951,  9.53968464,  9.63292011, ..., 14.85953652,
         14.72059128, 14.57932831],
        [ 9.49615573,  9.43106199,  9.51639061, ..., 14.79577961,
         14.66900445, 14.50496867],
        ...,
        [ 1.33641754,  1.3431475 ,  1.34435628, ...,  1.84525229,
          2.06413202,  2.49548907],
        [ 1.33569733,  1.3423575 ,  1.34386781, ...,  1.84525229,
          2.06413202,  2.49548907],
        [ 1.33497711,  1.34165199,  1.34345516, ...,  1.84525229,
          2.06413202,  2.49548907]],

       [[ 9.62738752,  9.60155582,  9.69318342, ..., 14.89295089,
         14.74384667, 14.61212   ],
        [ 9.57817951,  9.53968464,  9.63292011, ..., 14.85953652,
         14.72059128, 14.57932831],
        [ 9.49615573,  9.43106199,  9.51639061, ..., 14.79577961,
         14.66900445, 14.50496867],
        ...,
        [ 1.33641754,  1.3431475 ,  1.34435628, ...,  

In [27]:
bathymetry

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]])

In [30]:
bathymetry2 = np.where(bathymetry == 0 , 1,0)

In [31]:
bathymetry2

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1]])

In [32]:
ds_input

<xarray.Dataset>
Dimensions:  (x: 38, z: 800, time: 312)
Coordinates:
  * x        (x) int64 0 1 2 3 4 5 6 7 8 9 10 ... 28 29 30 31 32 33 34 35 36 37
    lat      (x) float64 ...
    lon      (x) float64 ...
  * z        (z) float64 2.5 7.5 12.5 17.5 ... 3.988e+03 3.992e+03 3.998e+03
Dimensions without coordinates: time
Data variables:
    t_pot    (time, z, x) float64 9.627 0.0 9.693 0.0 0.0 ... 0.0 0.0 0.0 0.0